# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
# identify file path
weather_csv = '../main_script/weather_data.csv'

# turn csv into dataframe
weather_df = pd.read_csv(weather_csv)

weather_df.head()


,Name,Latitude,Longitude,Temperature(F),Wind Speed(mph),Cloud Coverage,Humidity
0,Fort Smith,35.3859,-94.3986,61.070,1.05,1,29
1,Taquari,-29.7997,-51.8644,69.278,1.70,24,85
2,Grand Gaube,-20.0064,57.6608,81.266,3.13,100,79
3,Albany,42.6001,-73.9662,24.278,3.86,43,52
4,Port Alfred,-33.5906,26.8910,68.270,4.02,100,90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
gmaps.configure(api_key=g_key)

# convert humidity to float, drop NaN values
weather_df = weather_df.dropna()
humidity = weather_df['Humidity'].astype(float)

# store lattitude and longitude together
locations = weather_df[['Latitude', 'Longitude']].astype(float)

# create humidity heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, 
                                     weights = humidity, 
                                     dissipating = False, 
                                     max_intensity = max(weather_df['Humidity']), 
                                     point_radius = 2)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [3]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
